In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16


In [3]:

# Load the MNIST dataset and convert images to 32x32x3
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = tf.repeat(train_images[..., tf.newaxis], 3, -1)
test_images = tf.repeat(test_images[..., tf.newaxis], 3, -1)
train_images = tf.image.resize(train_images, (32, 32))
test_images = tf.image.resize(test_images, (32, 32))


In [6]:

# Load the VGG16 model with pretrained weights
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the pretrained layers
for layer in vgg16.layers:
    layer.trainable = False

# Build the model by adding the VGG16 model and a classifier on top
model = Sequential([
    vgg16,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 14,781,642
Trainable params: 66,954
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))


1875/1875 [==============================] - 332s 177ms/step - loss: 0.8426 - accuracy: 0.7831 - val_loss: 0.5068 - val_accuracy: 0.8221


In [8]:

# Test the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Float model Test accuracy:', test_acc)


313/313 [==============================] - 48s 153ms/step - loss: 0.5068 - accuracy: 0.8221
Float model Test accuracy: 0.8220999836921692


In [15]:
import tensorflow_model_optimization as tfmot
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

def apply_quantization_to_dense(layer):
    if type(layer) in [keras.layers.Dense]:
        return quantize_annotate_layer(layer)
    return layer

annotated_model = tf.keras.models.clone_model(
        model,
        clone_function=apply_quantization_to_dense,
    )

qat_model = tfmot.quantization.keras.quantize_apply(annotated_model)
qat_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

y_test = keras.utils.to_categorical(test_labels, 10)
_, qat_acc = qat_model.evaluate(test_images, y_test)

print('Quantized model Test accuracy :', qat_acc)

313/313 [==============================] - 42s 133ms/step - loss: 0.8368 - accuracy: 0.7182
Quantized model Test accuracy : 0.7182000279426575


In [17]:
print('Float model Test accuracy \t:', test_acc)
print('Quantized model Test accuracy \t:', qat_acc)

Float model Test accuracy 	: 0.8220999836921692
Quantized model Test accuracy 	: 0.7182000279426575
